# Exploratory Analisys

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

from database.mongo import Mongo
from pytorch_common.util import LoggerBuilder

import numpy as np
# import cudf as pd
import pandas as pd
import torch

from data.missing import MissingsManager

# import dtale

In [3]:
LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

## Load datasets

1. Import data to **movies** mongodb database:

In [4]:
DATASETS_PATH   = '../datasets'

movies          = pd.read_json(f'{DATASETS_PATH}/movies_v4.json')
interactions    = pd.read_json(f'{DATASETS_PATH}/ratings_tags_v1.json')

In [5]:
interactions = interactions.drop('_id', axis=1)

movies['id']  = movies['_id']
movies  = movies.drop(['_id', 'tmdb_id', 'imdb_id'], axis=1)

In [6]:
interactions.dtypes

rating              float64
timestamp    datetime64[ns]
user_id               int64
movie_id              int64
tags                 object
dtype: object

In [7]:
movies.dtypes

title                 object
genres                object
for_adults            object
budget                 int64
original_language     object
overview              object
poster                object
release               object
popularity           float64
vote_mean            float64
vote_count           float64
tags                  object
id                     int64
dtype: object

In [8]:
interactions.head()

,rating,timestamp,user_id,movie_id,tags
0,4.0,2015-08-13 13:23:59,3,260,"[classic, sci-fi]"
1,4.5,2019-11-16 22:33:10,4,1732,"[dark comedy, great dialogue]"
2,3.5,2019-11-16 22:30:31,4,7569,[so bad it's good]
3,5.0,2019-11-16 21:10:28,4,44665,[unreliable narrators]
4,5.0,2019-11-16 21:07:21,4,115713,"[artificial intelligence, philosophical, tense]"


In [9]:
movies.head()

,title,genres,for_adults,budget,original_language,overview,poster,release,popularity,vote_mean,vote_count,tags,id
0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",False,30000000,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,21.946943,7.7,5415.0,"[2009 reissue in stereoscopic 3-d, 3d, 55 movi...",1
1,Jumanji,"[Adventure, Children, Fantasy]",False,65000000,en,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,17.015539,6.9,2413.0,"[adaptation of book, adapted from:book, advent...",2
2,Sabrina,"[Comedy, Romance]",False,58000000,en,An ugly duckling having undergone a remarkable...,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,6.677277,6.2,141.0,"[as good maybe better than original, based on ...",7
3,Waiting to Exhale,"[Comedy, Drama, Romance]",False,16000000,en,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,3.859495,6.1,34.0,"[based on novel or book, characters, chick fli...",4
4,Father of the Bride Part II,[Comedy],False,0,en,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,8.387519,5.7,173.0,"[aging, baby, clv, comedy, confidence, contrac...",5


In [10]:
# dtale.show(movies.to_pandas())

In [11]:
# dtale.show(interactions.to_pandas())

## Check Missing values

Since there are prqactically no missing values, we remove this:

In [12]:
mm = MissingsManager() \
    .add_rule_fn('budget', lambda v: v == 0) \
    .add_rule_fn('vote_mean', lambda v: v == 0) \
    .add_rule_fn('vote_count', lambda v: v == 0)

movies.pipe(mm.report)

,Column,Percent (%),Count,rule,indexes
0,budget,0.772227,27143,CustomMissingFnRule,"[4, 8, 10, 11, 14, 15, 17, 18, 21, 25, 26, 27,..."
1,poster,0.000085,3,ColumnMissingValueRule,"[17450, 24442, 28570]"
2,release,0.000085,3,ColumnMissingValueRule,"[17450, 24442, 28570]"
3,popularity,0.000085,3,ColumnMissingValueRule,"[17450, 24442, 28570]"
4,vote_mean,0.048024,1688,CustomMissingFnRule,"[100, 123, 126, 151, 363, 365, 378, 539, 545, ..."
5,vote_count,0.045833,1611,CustomMissingFnRule,"[100, 123, 126, 151, 363, 365, 378, 545, 547, ..."


In [13]:
movies2 = movies.pipe(mm.remove_rows, max_percent=0.06)

2022-04-17 17:29:10,539 - INFO - Remove rows with missing <= 0.06 %


In [14]:
movies2.pipe(mm.report)

,Column,Percent (%),Count,rule,indexes
0,budget,0.762389,25508,CustomMissingFnRule,"[4, 8, 10, 11, 14, 15, 17, 18, 21, 25, 26, 27,..."


In [15]:
movies3 = movies2.pipe(mm.remove_columns, max_percent=0.4)

2022-04-17 17:29:10,826 - INFO - Remove columns with missing >= 0.4 %


In [16]:
movies3.pipe(mm.report)

,Column
0,Not found columns with missing values


In [19]:
movies3.head()

,title,genres,for_adults,original_language,overview,poster,release,popularity,vote_mean,vote_count,tags,id
0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",False,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,21.946943,7.7,5415.0,"[2009 reissue in stereoscopic 3-d, 3d, 55 movi...",1
1,Jumanji,"[Adventure, Children, Fantasy]",False,en,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,17.015539,6.9,2413.0,"[adaptation of book, adapted from:book, advent...",2
2,Sabrina,"[Comedy, Romance]",False,en,An ugly duckling having undergone a remarkable...,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,6.677277,6.2,141.0,"[as good maybe better than original, based on ...",7
3,Waiting to Exhale,"[Comedy, Drama, Romance]",False,en,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,3.859495,6.1,34.0,"[based on novel or book, characters, chick fli...",4
4,Father of the Bride Part II,[Comedy],False,en,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,8.387519,5.7,173.0,"[aging, baby, clv, comedy, confidence, contrac...",5


In [18]:
interactions.pipe(mm.report)

,Column
0,Not found columns with missing values


## Variables